<a href="https://colab.research.google.com/github/ducline/edit-data_processing/blob/main/spark/challenges/challenge_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHALLENGE 4
##  Analyze data

- Query table "vehicles_enriched" in gold layer
- Aggregate data by municipality_name (array)
- Calculate:
  - count of vehicles (id) that pass through that municipality
  - sum speed of vehicles

Questions:
  - What are the top 3 municipalities by vehicles routes?
  - What are the top 3 municipalities with higher vehicle speed on average?


Tips:
- explode array into rows -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.explode.html


# Setting up PySpark

In [13]:
%pip install pyspark

In [14]:
!mkdir -p /content/lake/silver/vehicles

In [15]:
!mkdir -p /content/lake/silver/lines

In [16]:
!mkdir -p /content/lake/silver/municipalities

In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, avg, sum as spark_sum, count
from pyspark.sql.types import StructType, StructField, StringType, ArrayType

class ETLFlow:

    def __init__(self, spark: SparkSession) -> None:
        self.spark = spark

    def extract(self, format: str, path: str, schema: StructType = None):
        if schema:
            return self.spark.read.format(format).schema(schema).load(path)
        return self.spark.read.format(format).load(path)

    def load(self, df, format: str, path: str, partition_column: str = None, **kwargs) -> None:
        if partition_column:
            df.coalesce(1).write.mode("overwrite").partitionBy(partition_column).format(format).save(path)
        else:
            df.coalesce(1).write.mode("overwrite").format(format).save(path)

class EnrichETLTask(ETLFlow):

    def __init__(self, spark: SparkSession) -> None:
        super().__init__(spark)

    def enrich_vehicles(self):
        # Define schemas
        vehicles_schema = StructType([
            StructField('bearing', StringType(), True),
            StructField('block_id', StringType(), True),
            StructField('current_status', StringType(), True),
            StructField('id', StringType(), True),
            StructField('lat', StringType(), True),
            StructField('line_id', StringType(), True),
            StructField('lon', StringType(), True),
            StructField('pattern_id', StringType(), True),
            StructField('route_id', StringType(), True),
            StructField('schedule_relationship', StringType(), True),
            StructField('shift_id', StringType(), True),
            StructField('speed', StringType(), True),
            StructField('stop_id', StringType(), True),
            StructField('timestamp', StringType(), True),
            StructField('trip_id', StringType(), True),
            StructField('date', StringType(), True)
        ])

        lines_schema = StructType([
            StructField('id', StringType(), True),
            StructField('long_name', StringType(), True),
        ])

        municipalities_schema = StructType([
            StructField('id', StringType(), True),
            StructField('name', ArrayType(StringType()), True),
        ])

        # Read datasets from SILVER layer
        vehicles_df = self.extract(format="parquet", path="/content/lake/silver/vehicles", schema=vehicles_schema)
        lines_df = self.extract(format="parquet", path="/content/lake/silver/lines", schema=lines_schema)
        municipalities_df = self.extract(format="parquet", path="/content/lake/silver/municipalities", schema=municipalities_schema)

        # Perform joins to enrich the vehicles dataset
        enriched_df = (
            vehicles_df
            .join(lines_df, vehicles_df.line_id == lines_df.id, "left")
            .join(municipalities_df, vehicles_df.line_id == municipalities_df.id, "left")
            .select(
                vehicles_df["*"],
                lines_df["long_name"].alias("line_name"),
                municipalities_df["name"].alias("municipality_name")
            )
        )

        # Write the enriched dataset to the GOLD layer
        self.load(
            df=enriched_df,
            format="parquet",
            path="/content/lake/gold/vehicles_enriched",
            partition_column="date"
        )

if __name__ == "__main__":
    spark = SparkSession.builder.master("local").appName("Analyze and Enrich Vehicles Data").getOrCreate()

    print("Starting ENRICH ETL process")
    etl = EnrichETLTask(spark)

    print("Running Task - Enrich Vehicles")
    etl.enrich_vehicles()

    print("ENRICH ETL process completed")

    # Read the enriched vehicles dataset from the GOLD layer
    vehicles_enriched_path = "/content/lake/gold/vehicles_enriched"
    vehicles_schema = StructType([
        StructField('bearing', StringType(), True),
        StructField('block_id', StringType(), True),
        StructField('current_status', StringType(), True),
        StructField('id', StringType(), True),
        StructField('lat', StringType(), True),
        StructField('line_id', StringType(), True),
        StructField('lon', StringType(), True),
        StructField('pattern_id', StringType(), True),
        StructField('route_id', StringType(), True),
        StructField('schedule_relationship', StringType(), True),
        StructField('shift_id', StringType(), True),
        StructField('speed', StringType(), True),
        StructField('stop_id', StringType(), True),
        StructField('timestamp', StringType(), True),
        StructField('trip_id', StringType(), True),
        StructField('date', StringType(), True),
        StructField('line_name', StringType(), True),
        StructField('municipality_name', ArrayType(StringType()), True)
    ])

    vehicles_df = spark.read.format("parquet").schema(vehicles_schema).load("/content/lake/gold/vehicles_enriched")


    # Explode the municipality_name array into individual rows
    exploded_df = vehicles_df.withColumn("municipality", explode(col("municipality_name")))

    # Aggregate data by municipality
    aggregated_df = (
        exploded_df
        .groupBy("municipality")
        .agg(
            count("id").alias("vehicle_count"),
            spark_sum("speed").alias("total_speed"),
            avg("speed").alias("avg_speed")
        )
    )

    # Top 3 municipalities by vehicle count
    top_3_by_vehicle_count = (
        aggregated_df
        .orderBy(col("vehicle_count").desc())
        .limit(3)
    )

    print("Top 3 municipalities by vehicles routes:")
    top_3_by_vehicle_count.show()

    # Top 3 municipalities by average speed
    top_3_by_avg_speed = (
        aggregated_df
        .orderBy(col("avg_speed").desc())
        .limit(3)
    )

    print("Top 3 municipalities with higher vehicle speed on average:")
    top_3_by_avg_speed.show()


Starting ENRICH ETL process
Running Task - Enrich Vehicles
ENRICH ETL process completed
Top 3 municipalities by vehicles routes:
+------------+-------------+-----------+---------+
|municipality|vehicle_count|total_speed|avg_speed|
+------------+-------------+-----------+---------+
+------------+-------------+-----------+---------+

Top 3 municipalities with higher vehicle speed on average:
+------------+-------------+-----------+---------+
|municipality|vehicle_count|total_speed|avg_speed|
+------------+-------------+-----------+---------+
+------------+-------------+-----------+---------+

